# 1)**import module**

In [ ]:
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **2) Dataset build**

In [ ]:
project_folder = '/content/drive/MyDrive/Project3'

import os

image = []
label = []

for subdir, _, files in os.walk(project_folder):
    for file in files:
        if file.endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(subdir, file)
            image.append(image_path)

            label_name = os.path.basename(subdir)
            label.append(label_name)

In [ ]:
BATCH_SIZE = 1 #변경 금지

TRAINING_image = []
TRAINING_label = []
TEST_image = []
TEST_label = []

for i in range(0,80):
  for j in range(0,20):
    for k in range(0,2):
      TRAINING_image.append(image[200*j+i+k])
      TRAINING_label.append(label[200*j+i+k])

for i in range(80, 100):
  for j in range(0,20):
    for k in range(0,2):
      TEST_image.append(image[200*j+i+k])
      TEST_label.append(label[200*j+i+k])


import random

def shuffle_images_labels(image_list, label_list):
    combined_list = list(zip(image_list, label_list))  # 이미지와 라벨을 쌍으로 묶음
    for i in range(1, len(combined_list)-1, 2):
        if random.random() < 0.3:  # 50% 확률로 스왑 수행
            combined_list[i], combined_list[i+1] = combined_list[i+1], combined_list[1]
    return zip(*combined_list)


TRAINING_image, TRAINING_label = shuffle_images_labels(TRAINING_image, TRAINING_label)

# 코드 문제점 개선
- 실제로는 30% 확률로 스왑을 수행하고 있다.
- i와 i+1을 swap하는 게 아니었을까 생각이 들어 수정해봅니다.

```
if random.random() < 0.5:  # 50% 확률로 스왑 수행
            combined_list[i], combined_list[i+1] = combined_list[i+1], combined_list[i]  # 스왑 수행
```




In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('RGB')
        image = transforms.Resize((224, 224))(image)
        image = transforms.ToTensor()(image)

        return image, label

    #Renoir 1.0에서는 Randomcrop((224,224))을 사용하였으나,
    #이미지가 모델에 입력되기도 전에 정보 손실이 생기지 않게 하기 위해
    #Renoir 2.0에서는 Resize((224,224)) 사용

train_dataset = CustomDataset(TRAINING_image, TRAINING_label)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE)
test_dataset = CustomDataset(TEST_image, TEST_label)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [ ]:
# __getitem__ 함수는 주어진 경로의 이미지를 불러오고, 크기를 224x224로 Resize하고, Tensor로 변환한 후 레이블과 함께 반환합니다.
# DataLoader를 사용하여 배치 단위로 데이터를 나누어 학습할 수 있게 준비합니다.

# **3) Model Define**

In [ ]:
# GoogLeNet 모델을 사용하여 얼굴 유사도 비교를 수행하는 샴 네트워크 구조를 기반으로 한 모델을 학습합니다.
# GoogLeNet 아키텍처를 구현하며, 기본적으로 Inception 모듈을 사용하여 네트워크의 다양한 계층을 쌓습니다.
# Inception Module : 다양한 크기의 커널(1x1, 3x3, 5x5)과 Max Pooling을 결합해 다양한 스케일의 특징을 추출
# 모델의 마지막은 AvgPool2d로 크기를 1x1로 줄이고, 결과는 벡터 형태로 반환합니다.

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor

def conv_1(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, 1, 1),
      nn.ReLU(),
  )
  return model

def conv_1_3(in_dim,mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim,out_dim,3,1,1),
        nn.ReLU()
    )
    return model

def conv_1_5(in_dim,mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim,out_dim,5,1,2),
        nn.ReLU()
    )
    return model

def max_3_1(in_dim,out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
        nn.Conv2d(in_dim,out_dim,1,1),
        nn.ReLU(),
    )
    return model

class inception_module(nn.Module):
    def __init__(self,in_dim,out_dim_1,mid_dim_3,out_dim_3,mid_dim_5,out_dim_5,pool_dim):
        super(inception_module,self).__init__()

        self.conv_1 = conv_1(in_dim,out_dim_1)
        self.conv_1_3 = conv_1_3(in_dim,mid_dim_3,out_dim_3)
        self.conv_1_5 = conv_1_5(in_dim,mid_dim_5,out_dim_5)
        self.max_3_1 = max_3_1(in_dim,pool_dim)

    def forward(self,x):
        out_1 = self.conv_1(x)
        out_2 = self.conv_1_3(x)
        out_3 = self.conv_1_5(x)
        out_4 = self.max_3_1(x)

        output = torch.cat([out_1,out_2,out_3,out_4],1)
        return output


In [ ]:
#nn.Conv2d(input_chennel, output_channel, kernel_size, stride, padding)
#nn.MaxPool2d(kernel_size, stride, padding)

class GoogLeNet(nn.Module):
    def __init__(self, base_dim):
        super(GoogLeNet, self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            nn.MaxPool2d(3,2,1),
            nn.Conv2d(base_dim,base_dim*3,3,1,1),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            inception_module(base_dim*3,64,96,128,16,32,32),
            inception_module(base_dim*4,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_3 = nn.Sequential(
            inception_module(480,192,96,208,16,48,64),
            inception_module(512,160,112,224,24,64,64),
            inception_module(512,128,128,256,24,64,64),
            inception_module(512,112,144,288,32,64,64),
            inception_module(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_4 = nn.Sequential(
            inception_module(832,256,160,320,32,128,128),
            inception_module(832,384,192,384,48,128,128),
            nn.AvgPool2d(7,1), #nn.AvgPool2d(7,1) output size : 1x1x1024
        )

    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = out.view(out.size(0), -1)
        return out  #self.layer_4를 통과한 tensor return

# **4)Model initialization**

In [ ]:
import torch.nn.init as init
import torch.nn as nn

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GoogLeNet(base_dim=64)
model = model.to(DEVICE)

loss = nn.BCELoss()
optimizer =torch.optim.SGD(model.parameters(), lr = 0.005,momentum = 0.9, weight_decay = 0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.04)

# **5) TEST function define**

In [ ]:
def test(test_loader, DEVICE):
  correct_pred = 0

  for i, (_image1, _label1) in enumerate(test_loader):
    image1 = _image1.to(DEVICE)
    label1 = _label1[0]
    vector1_tensor = model(image1)

    if (i == 0):  # Exception Case
      label2 = label1
      vector2_tensor = vector1_tensor

    target_vector = [label1 == label2]
    target_tensor = torch.tensor(target_vector).float()
    target_tensor = target_tensor.to(DEVICE)

    similarity =  F.cosine_similarity(vector1_tensor, vector2_tensor, dim= -1)
    scaled_similarity = torch.sigmoid(similarity)

    if label1 == label2 and scaled_similarity.item() > 0.8:
        correct_pred += 1
    elif label1 != label2 and scaled_similarity.item() < 0.8:
        correct_pred += 1

    #연산량 감소를 위한 텐서 재활용
    label2 = label1
    vector2_tensor = vector1_tensor.detach()

  return correct_pred

# **6) Training**

In [ ]:
# 손실 함수로는 **Binary Cross Entropy (BCELoss)**를 사용하고
# 최적화 기법으로는 SGD(Stochastic Gradient Descent)를 사용합니다.
# 학습률 감소를 위해 Scheduler를 사용해 매 8 에포크마다 학습률을 감소시킵니다.

In [ ]:
# 학습 중 두 개의 이미지 벡터를 구하고, 이 벡터 간의 코사인 유사도를 계산합니다.
# Cosine Similarity를 기반으로 두 이미지가 동일한지 아닌지 판단하고, 그에 따라 맞는 예측을 카운트합니다.
# 손실 함수는 scaled_similarity와 목표 벡터 간의 차이를 최소화하는 방식으로 동작합니다.

In [ ]:
import torch.nn.functional as F
import time

EPOCH = 50

start_time = time.time()
train_acc_lst, test_acc_lst = [],[]

for epoch in range(EPOCH):
  model.train()
  correct_pred, num_examples, test_acc = 0, len(train_loader), 0

  for i, (_image1, _label1) in enumerate(train_loader):
    optimizer.zero_grad()

    image1 = _image1.to(DEVICE)
    label1 = _label1[0]
    vector1_tensor = model(image1)

    if (i == 0):  # Exception Case
      label2 = label1
      vector2_tensor = vector1_tensor

    target_vector = [label1 == label2]
    target_tensor = torch.tensor(target_vector).float()
    target_tensor = target_tensor.to(DEVICE)

    similarity =  F.cosine_similarity(vector1_tensor, vector2_tensor, dim= -1)
    scaled_similarity = torch.sigmoid(similarity)

    if label1 == label2 and scaled_similarity.item() > 0.8:
        correct_pred += 1
    elif label1 != label2 and scaled_similarity.item() < 0.8:
        correct_pred += 1

    cost= loss(scaled_similarity, target_tensor)
    cost.backward()
    optimizer.step()

    #연산량 감소를 위한 텐서 재활용
    label2 = label1
    vector2_tensor = vector1_tensor.detach()

    if not i % 40:
      print (f'Epoch: {epoch+65:03d} | '
            f'Batch {i:03d}/{len(train_loader):03d} |'
             f' Cost: {cost:.4f}')

  test_acc = test(test_loader, DEVICE)

  PATH = f"/content/drive/MyDrive/model_weights_epoch_{epoch}.pth"  # 에포크별로 파일 이름 지정
  torch.save(model.state_dict(), PATH)

  print('training accuracy : ', (correct_pred/len(train_loader))*100,'% | test accuracy : ', (test_acc/len(test_loader))*100, '%')

  elapsed = (time.time() - start_time)/60
  print(f'Time elapsed: {elapsed:.2f} min')

elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')